In [1]:
import numpy as np
import pandas as pd
import os, random, shutil, sys, time
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sys

#mac
util_path = "/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/code/latest/util"

#constance
'''args = sys.argv[0:]
py_file = args[0]

code_location = os.path.dirname(os.path.abspath(py_file))
index_of_latest = code_location.index('latest')
util_path = os.path.join(code_location[:index_of_latest], "latest", "util")
'''

sys.path.insert(0, util_path)

In [2]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/code/latest/2_bmd/2_bmd_cal_itself/latest/Bree/LDH


In [3]:
# new for LDH
complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/Bree/LDH/DN_edited/BHT/input/BHT.csv'
df_test_dose_response = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)
display(df_test_dose_response.head())

,dose,num_affected,total_num
0,0,0.257640,100
1,10,0.000000,100
2,25,0.204187,100
3,50,0.370192,100
4,75,2.101629,100


In [4]:
import generate_dose_response as gdr
import BMD_BMDL_estimation as bmdest
import Plot_Save as ps

In [5]:
# Fit dose response models
start_time = time.time()

df_test_dose_response = df_test_dose_response.sort_values(by=['dose'])

model_predictions = bmdest.analyze_dose_response_data(df_test_dose_response)
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("BMD calculation is done, it took:"+str(time_took))

BMD calculation is done, it took:2.4 seconds


In [6]:
print (model_predictions)

            Model  Chi-squared         p-val         AIC      BMD10  \
0        logistic    19.535770  1.526811e-03  156.692547  87.116028   
1           gamma     2.196970  6.995840e-01  159.057589  84.623453   
2         weibull          NaN           NaN         NaN        NaN   
3    log_logistic     0.492845  9.741937e-01  156.632893  89.287652   
4          probit          NaN           NaN         NaN        NaN   
5      log_probit          NaN           NaN         NaN        NaN   
6    multistage_2          NaN           NaN         NaN        NaN   
7  quantal_linear   149.155333  2.019823e-30  381.383544  26.799965   

      BMDL10       BMD50                                   Scaled Residuals  \
0  82.151746  104.544236  [-4.252327979464805, 0.09480594565684891, -0.8...   
1  80.791342  116.608167  [-0.25569900920415956, 0.3959728515540838, -0....   
2        NaN         NaN                [nan, nan, nan, nan, nan, nan, nan]   
3  84.534675  108.955991  [-0.10770466311601

In [7]:
# Select best model
selected_model_params = bmdest.select_model(model_predictions)
print (selected_model_params)

{'model': 3    log_logistic
Name: Model, dtype: object, 'no_unique_model_found_flag': 0, 'model_select_flag': 0, 'pval_thresh_flag': 0}


In [8]:
#bmd_feasibility_flag = gdr.BMD_feasibility_analysis(df_test_dose_response)
#print ("bmd_feasibility_flag:"+str(bmd_feasibility_flag))

In [9]:
chemical_id = "BHT"
end_point = "LDH"
bmd_feasibility_flag = 2

# Check if unique model is found
unique_model_flag = selected_model_params['no_unique_model_found_flag']

if(unique_model_flag == 0):
    ps.save_results_good_data_unique_model(df_test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
else:
    bmd_analysis_flag = selected_model_params['model_select_flag']
    if(bmd_analysis_flag == 1):
        ps.save_results_poor_data_or_no_convergence(df_test_dose_response, bmd_feasibility_flag, str(chemical_id), end_point, selected_model_params)
    else:
        ps.save_results_good_data_nounique_model(df_test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)